In [1]:
import cv2
import numpy as np
from numpy.core.defchararray import join, mod
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import os
import torch
from torch._C import device
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
from torchvision import transforms as torchtrans
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
##Hyper Parameters##
batch_size=5
num_classes=2
num_epoch = 10
lr = 0.01
momentum = 0.9

#**Data Preprocessing** **bold text**

In [4]:
IMAGES_PATH = './drive/MyDrive/Colab Notebooks/input/Images'
LABELS_PATH = './drive/MyDrive/Colab Notebooks/input/Labels'
VERSION = '1'
MODEL_SAVE_PATH = './WeaponDetection'+ VERSION + ".pt"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
class Images(Dataset):
    def __init__(self,imgs_path,labels_path):

        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.img_name = [img for img in sorted(os.listdir(self.imgs_path))]
        self.label_name = [label for label in sorted(os.listdir(self.labels_path))]

    def __getitem__(self,idx):

        image_path = os.path.join(self.imgs_path,str(self.img_name[idx]))
        img = cv2.imread(image_path)

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = img_rgb/255
        img_res = torch.as_tensor(img_res).to(device)
        img_res = img_res.permute(2, 0, 1)

        label_name = self.img_name[idx][:-4] + "txt"
        label_path = os.path.join(self.labels_path,str(label_name))
        with open(label_path, 'r') as label_file:
            l_count = int(label_file.readline())
            box = []
            for i in range(l_count):
                box.append(list(map(int, label_file.readline().split())))

        target={}
        target["boxes"] = torch.as_tensor(box).to(device)
        area = []
        for i in range(len(box)):

            a = (box[i][2] - box[i][0]) * (box[i][3] - box[i][1])
            area.append(a)
        target["area"] = torch.as_tensor(area).to(device)
        labels = []
        for i in range(len(box)):
            labels.append(1)

        target["image_id"] = torch.as_tensor([idx]).to(device)
        target["labels"] = torch.as_tensor(labels, dtype = torch.int64).to(device)


        return img_res,target

    def __len__(self):
        return len(self.img_name)


def collate_fn(batch):
    return tuple(zip(*batch))

In [7]:
gun_data = Images(IMAGES_PATH, LABELS_PATH)
train_data = DataLoader(gun_data, batch_size=5,
                       shuffle=True, num_workers=0, collate_fn=collate_fn)

# **Creating the Model**

In [10]:
class Model():
  def __init__(self,num_classes,epochs=30,lr=0.01, momentum=0.9):
    #creating faster rcnn model
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
        in_features, num_classes)
    model.to(device)
    self.model = model

    self.epochs = epochs
    self.lr = lr
    self.momentum = momentum

  def train(self,train_data):
    param = [param for param in self.model.parameters() if param.requires_grad]

    optimizer = torch.optim.SGD(param,lr=self.lr,momentum=self.momentum)

    for epoch in range(self.epochs):
      tot_loss = 0
      self.model.train()
      for img, target in train_data:
          loss_dict = self.model(img, target)
          loss = sum(loss for loss in loss_dict.values())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          tot_loss += loss
      print("eppch:{},loss:{}".format(epoch, tot_loss))

  def save(self,file_path):
    torch.save(self.model, file_path)


# **Training the model**

In [ ]:
model = Model(2,num_epoch,lr,momentum)
model.train(train_data)
model.save()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


eppch:0,loss:16.60452651977539
eppch:1,loss:11.904146194458008
eppch:2,loss:8.87230110168457
eppch:3,loss:7.0211920738220215
eppch:4,loss:6.050671100616455
eppch:5,loss:5.6785664558410645
eppch:6,loss:4.997640609741211
eppch:7,loss:4.763161659240723
eppch:8,loss:4.346904754638672
